In [ ]:
%pip install --index-url http://74.82.29.209:8090 nebula3_database==0.2.2 --trusted-host 74.82.29.209

In [ ]:
%pip install -U sentence-transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from database.arangodb import DatabaseConnector
import torch
import weaviate
import json
from sentence_transformers import SentenceTransformer, util
import uuid

In [ ]:
client = weaviate.Client("http://64.71.146.93:8080") # <== if you use Docker-compose
gdb = DatabaseConnector()
db = gdb.connect_db('prodemo')
schema = client.schema.get()
print(json.dumps(schema))

In [ ]:
st_model = SentenceTransformer('all-mpnet-base-v2', cache_folder="all_mpnet_base")

In [ ]:
# we will create the class "Author" and the properties
# from the basics section of this guide
class_obj = {
    "class": "Weak_Sim_Intent", # <= note the capital "A".
    "description": "Add weak intent similarity",
    "vectorizer": "none",
    "properties": [
        {
            "dataType": [
                "int"
            ],
            "description": "Frame_nmb",
            "name": "frame_nmbr",
        },
        {
            "dataType": [
                "string"
            ],
            "description": "movie_id",
            "name": "movie_id"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "url",
            "name": "url"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "caption",
            "name": "caption"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "intent",
            "name": "intent"
        }
    ]
}

# add the schema
client.schema.create_class(class_obj)

# get the schema
schema = client.schema.get()

# print the schema
print(json.dumps(schema, indent=4))

In [ ]:
gr = []
query = 'FOR doc IN  llm_weak_sim_intent RETURN doc'
cursor = db.aql.execute(query)
for line in cursor:
    for ws in line['intents']:
        data_object = {"movie_id": line['movie_id'], "frame_nmbr": line['frame_num'], 
        "url": line['url'], "caption": line['candidate'], "intent": ws}
        print(data_object)
        vector = st_model.encode(ws, convert_to_tensor=True)
        uuid_ = uuid.uuid1()
        print(uuid_)
        #print(vector)
        #input()
        client.data_object.create(
        data_object,
        "Weak_Sim_Intent",
        str(uuid_),
        vector = vector # supported types are `list`, 'numpy.ndarray`, `torch.Tensor` and `tf.Tensor`. Make sure the length matches with your Weaviate settings.
        )    

In [45]:
testVector = st_model.encode("arrest the citizen", convert_to_tensor=True)
nearVector = {"vector": testVector, 'certainty': 0.70}
#print(nearVector)
client.query.get("Weak_Sim_Intent",["caption", "url", "_additional {certainty}"]).with_near_vector(nearVector).do()

{'data': {'Get': {'Weak_Sim_Intent': [{'_additional': {'certainty': 0.7170430421829224},
     'caption': 'A group of people are sitting on a stage. They are all wearing black. One woman has a microphone in front of her. There is a man sitting in the middle of the group. There is a woman to the right of the group. She is clapping.',
     'url': 'http://74.82.29.209:9000/datasets/media/frames/video7014/frame0091.jpg'},
    {'_additional': {'certainty': 0.7128855884075165},
     'caption': 'This image appears to be taken in a music or television studio. Two men are sitting in chairs, facing each other. One man is wearing a black hoodie and the other is wearing a white collared shirt.',
     'url': 'http://74.82.29.209:9000/datasets/media/frames/video7014/frame0155.jpg'},
    {'_additional': {'certainty': 0.7128855884075165},
     'caption': 'A group of people are sitting on a stage. They are all wearing black. One woman has a microphone in front of her. There is a man sitting in the middl

In [48]:
client.query.get("Weak_Sim_Intent","vector").do()

{'errors': [{'locations': [{'column': 22, 'line': 1}],
   'message': 'Cannot query field "vector" on type "Weak_Sim_Intent".',
   'path': None}]}